##### 1.환경변수 설정

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

sk
Fy
tvly


In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain_openai import ChatOpenAI
# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain.document_loaders import TextLoader
import re
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))
def split_menu_items(document):
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'

    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 분리된 각 메뉴 항목을 저장할 빈 리스트를 만듭니다.
    menu_documents = []
    
    # enumerate()를 사용하여 각 메뉴 항목과 그 인덱스(i)를 순회합니다. 인덱스는 1부터 시작합니다.
    for i, item in enumerate(menu_items, 1):
        # 각 메뉴 항목 텍스트에서 첫 번째 줄을 가져와 '숫자.' 부분을 제거하고 메뉴 이름을 추출합니다.
        # 예: "1. 불고기 덮밥\n가격..." -> "불고기 덮밥"
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 LangChain Document 객체를 생성합니다.
        menu_doc = Document(
            # page_content에는 현재 메뉴 항목의 전체 텍스트를 할당합니다.
            page_content=item.strip(),
            # 문서와 관련된 메타데이터(부가 정보)를 딕셔너리 형태로 저장합니다.
            metadata={
                # 원본 문서의 출처(파일 경로 등)를 그대로 가져옵니다.
                "source": document.metadata['source'],
                # 메뉴의 순번을 저장합니다.
                "menu_number": i,
                # 추출한 메뉴 이름을 저장합니다.
                "menu_name": menu_name
            }
        )
        
        # 새로 생성한 Document 객체를 리스트에 추가합니다.
        menu_documents.append(menu_doc)
    
    # 모든 메뉴 항목이 Document 객체로 변환된 리스트를 반환합니다.
    return menu_documents

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, #List[Document]
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
cafe_db.save_local("./db/cafe_db")


# Retriever 생성
cafe_retriever = cafe_db.as_retriever(
    search_kwargs={'k': 4},
)


##### tool : db_search_cafe_func

In [6]:
from langchain_core.tools import tool
from typing import List                                         # 타입 힌트를 위한 List

# menu db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
특정 카페의 메뉴 정보, 가격, 또는 관련 세부 정보를 찾을 때 사용합니다. 
"""
    docs = cafe_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]

##### tool : tavily_search_func

In [7]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의  : tavily_search_func - StructuredTool 타입
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    #doc 변수가 dict 타입, ['title', 'url', 'content', 'score', 'raw_content']
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

##### tool : wiki_summary

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader
from pydantic import BaseModel, Field                           # 데이터 유효성 검사 및 스키마 정의를 위한 Pydantic
from typing import List                                         # 타입 힌트를 위한 List
# dedent() 함수를 가져옵니다. 이 함수는 문자열의 들여쓰기를 제거하는 역할을 합니다.
from textwrap import dedent

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

C:\Users\user\AppData\Local\Temp\ipykernel_2528\2892055845.py:38: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = summary_chain.as_tool(


In [10]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain


# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. "),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
tools =[db_search_cafe_func, tavily_search_func, wiki_summary]
# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
      

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("이 카페의 아메리카노의 가격과 특징은 무엇인가요?")

# 응답 출력 
pprint(response.content)


db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_dgxKt5t23ivKF0nry91nEjsM', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='e82b62e9-56f3-40f6-84b3-7e580fc0bf93', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 1, 'menu_name': '아메리카노'}, page_content='1. 아메리카노\\n   • 가격: ₩4,500\\n   • 주요 원료: 에스프레소, 뜨거운 물\\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.'), Document(id='14e72250-d54c-4057-9295-8c964ef3fa78', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 9, 'menu_name': '아이스 아메리카노'}, page_content='9. 아이스 아메리카노\\n   • 가격: ₩4,500\\n   • 주요 원료: 에스프레소, 차가운 물, 얼음\\n   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.'), Document(id='0d3e93b2-cf99-44e2-885e-cccde07a145c', metadata